## Great Plains Perennial Grasslands

In [1]:
import os
import arcpy

In [2]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
OutputWorkspace = r"D:\Blueprint\2022\Indicators\grasslands\PerennialForbsAndGrass\working"

In [3]:
# define raster used for cell size, extent, and snapping
SERaster= r"D:\Blueprint\2022\Masks\finalRegions_extent\BaseBlueprintExtent2022.tif"

In [4]:
# define additional inputs
nlcd= r"D:\Landcover\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"
BaseBlueprintSubregions = r"D:\Blueprint\2022\Masks\finalRegions_extent\DraftBaseBlueprintSubRgn.shp"
sourceData = r"D:\Blueprint\2022\Indicators\grasslands\vegetation-cover-v3-2020.tif"

In [5]:
# define indicator name
indicatorName = "GreatPlainsPerennialGrasslands.tif"

In [23]:
# define subregion masks to use
subregionList = "Edwards_Plateau_R.tif;High_Plains_and_Tablelands_R.tif;Plains_and_Timbers_R.tif;South_Texas_Plains_R.tif;Texas_Blackland_Prairies_R.tif"
#sub1 = r'D:\Blueprint\2022\Masks\FinalMasks\FinalSubregionsAndEdgesLargeRoads\Edwards_Plateau_R.tif'
#sub2 = r'D:\Blueprint\2022\Masks\FinalMasks\FinalSubregionsAndEdgesLargeRoads\High_Plains_and_Tablelands_R.tif'
#sub3 = r'D:\Blueprint\2022\Masks\FinalMasks\FinalSubregionsAndEdgesLargeRoads\Plains_and_Timbers_R.tif'
#sub4 = r'D:\Blueprint\2022\Masks\FinalMasks\FinalSubregionsAndEdgesLargeRoads\South_Texas_Plains_R.tif'
#sub5 = r'D:\Blueprint\2022\Masks\FinalMasks\FinalSubregionsAndEdgesLargeRoads\Texas_Blackland_Prairies_R.tif'

## Begin analysis

In [24]:
# Set the workspace 
arcpy.env.workspace = OutputWorkspace

In [26]:
# Merge rasters of the selected subregion boundaries
arcpy.management.MosaicToNewRaster(subregionList, OutputWorkspace, "IndicatorRegionsMerged.tif", number_of_bands=1)                   

<Result 'D:\\Blueprint\\2022\\Indicators\\grasslands\\PerennialForbsAndGrass\\working\\IndicatorRegionsMerged.tif'>

In [ ]:
# Remove band 4 (perennial forb and grass) from the multiband source data for indicator region. Update this if expanding to larger area
arcpy.management.MakeRasterLayer("vegetation-cover-v3-2020.tif", "perennialForbAndGrassSubregion.tif", '', "-107.32391860992 25.7404841784877 -94.5494038262524 37.2030851707942", 4)

In [6]:
# Make a raster to remove NLCD urban crop and water. They do that in the rangeland viewer for these data and it pulls out high values in less natural areas
out_raster = arcpy.sa.Con(nlcd, 0, 1, "Value = 11 OR Value = 21 OR Value = 22 OR Value = 23 OR Value = 24 OR Value = 82"); out_raster.save("nlcdFilter.tif")

In [6]:
# Now remove urban crop and water
out_raster = arcpy.sa.Times("perennialForbAndGrassSubregion.tif", "nlcdFilter.tif"); out_raster.save("indicatorFiltered_NoUrbanOrCrop.tif")

In [7]:
# reclassify to create binned version. After looking at known sites and aerials, a 5 class equal bin seems reasonable
out_raster = arcpy.sa.Reclassify("indicatorFiltered_NoUrbanOrCrop.tif", "Value", "0 20 1;21 40 2;41 60 3;61 80 4;81 100 5", "DATA"); out_raster.save("indicatorFiltered_NoUrbanOrCrop_5class.tif")

In [27]:
# clip to indicator extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SERaster, snapRaster=SERaster, cellSize=SERaster):
    out_raster = arcpy.sa.ExtractByMask("indicatorFiltered_NoUrbanOrCrop_5class.tif", "IndicatorRegionsMerged.tif"); out_raster.save(indicatorName)

In [28]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 5:
        return '5 = 81-100% perennial forbs and perennial grass'
    elif value == 4:
        return '4 = 61-80% perennial forbs and perennial grass'
    elif value == 3:
        return '3 = 41-60% perennial forbs and perennial grass'
    elif value == 2:
        return '2 = 21-40% perennial forbs and perennial grass'
    elif value == 1:
        return '1 = 0-20% perennial forbs and perennial grass'      
"""

In [29]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("GreatPlainsPerennialGrasslands.tif", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2022\\Indicators\\grasslands\\PerennialForbsAndGrass\\working\\GreatPlainsPerennialGrasslands.tif'>

In [30]:
# set code block for next step of adding RGB colors
codeblock = """
def Reclass1(Value):
	if Value == 5:
		return 58
	if Value == 4:
		return 105
	if Value == 3:
		return 153
	if Value == 2:
		return 203
	if Value == 1:
		return 253
	else:
		return 255
		
def Reclass2(Value):
	if Value == 5:
		return 67
	if Value == 4:
		return 111
	if Value == 3:
		return 157
	if Value == 2:
		return 205
	if Value == 1:
		return 254
	else:
		return 255
		
def Reclass3(Value):
	if Value == 5:
		return 17
	if Value == 4:
		return 58
	if Value == 3:
		return 100
	if Value == 2:
		return 144
	if Value == 1:
		return 190
	else:
		return 255
		
"""

In [31]:
# calculate Red field
arcpy.management.CalculateField("GreatPlainsPerennialGrasslands.tif", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("GreatPlainsPerennialGrasslands.tif", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("GreatPlainsPerennialGrasslands.tif", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2022\\Indicators\\grasslands\\PerennialForbsAndGrass\\working\\GreatPlainsPerennialGrasslands.tif'>